Тестовое задание

Цель: необходимо обработать сырые данные, поступившие от парсера.

Описание: в файле `hotels_parsing_result.xlsx` два листа - `platform1`, `platform2` с данными о гостиницах от двух разных платформ в одном регионе.

Поля:
- `id` - Идентификатор в БД;
- `create_time` - Дата создания;
- `title` - Название;
- `hotel_type_original` - Тип гостиницы;
- `city` - Город;
- `address` - Адрес;
- `rating` - Рейтинг;
- `rating_5` - Рейтинг по пятибалльной шкале;
- `review_count` - Количество отзывов;
- `star_rating` - Звёздность;
- `rooms_count` - Количество номеров;
- `contact_social` - Контакты соц. сетей;
- `description` - Описание;
- `email` - адрес строкой, несколько значений через запятую;
- `phone` - номер телефона строкой, несколько значений через запятую;
- `website` - сайты строкой, несколько значений через запятую;
- `uid` - Уникальный идентификатор гостиницы на платформе, не может быть разным у одной гостиницы, и не может повторятся у разных гостиниц, но в рамках одной платформы;
- `parsing_time` - Время сбора;
- `lat` - Широта;
- `lon` - Долгота.

Состав данных:
Записи могут повторяться, и самые актуальные данные идут в конце.
Если какое-то поле по конкретной гостинице пустое в последней записи, но встречалось ранее, то необходимо его взять из более ранних записей.
Поля телефона, email, сайтов могут содержать лишние символы, несколько записей и прочее.
Формат записи названия, адреса, типа гостиницы отличается в разных платформах.

Задача:
1. Собрать от каждой платформы финальный список гостиниц, в котором по каждой гостинице внутри платформы будет только одна запись с самыми актуальными и полными данными.
2. Почистить данные.
3. Поля телефона, email, сайтов распарсить и сохранить как списки в одинаковом формате (address@domen.org, 79234553322, domen.ru)
4. Вывести топ 10 по каждой платформе, по параметрам: больше всего телефонов, больше всего отзывов.
5. Вывести квадрат координат размером 1км на 1км, где больше всего гостиниц.
6. Задача со *, объединить данные от двух платформ, по критерию, который надо придумать. Например: вывести все гостиницы, которые есть в платформе 1 и нет в платформе 2; или вывести топ-10 гостиниц которые есть в обоих платформах, по суммарному количеству отзывов.

Требования:
1. Результат должен быть представлен Jupyter notebook.
2. Результат должен воспроизводится автоматически с нуля при повторном запуске, ручные правки должны быть учтены в коде.
3. Комментарии в коде приветствуются.


In [3]:
# Импорт библиотек
import pandas as pd
from fuzzywuzzy import process
import re
import numpy as np
from IPython.display import display

In [5]:
# Чтение данных из Excel файла
df = pd.read_excel('hotels_parsing_result.xlsx')

# Объединение данных с двух платформ
combined_data = pd.concat([df[df['platform'] == 'platform1'], df[df['platform'] == 'platform2']])

# Группировка данных по uid и выбор последнего значения для каждой гостиницы
grouped_data = combined_data.groupby('uid').tail(1)

KeyError: 'platform'

In [ ]:
# Чистка данных
def standardize_address(address):
# Преобразование адреса в стандартный формат
address = address.lower() # Приведение адреса к нижнему регистру
address = address.replace(" ", "") # Удаление пробелов
address = address.replace(",", "") # Удаление запятых
address = address.replace(".", "") # Удаление точек
return address

def clean_data(dataframe):
    # Чистка полей телефона, email, сайтов
    dataframe['phone'] = dataframe['phone'].apply(lambda x: ','.join(filter(None, re.split(r'\D+', str(x)))))
    dataframe['email'] = dataframe['email'].apply(lambda x: process.extractOne(x, dataframe['email'].tolist())[0] if x in dataframe['email'].tolist() else x)
    dataframe['website'] = dataframe['website'].apply(lambda x: x.lower().strip().replace('https://', '').replace('http://', '').split('/')[0])

return dataframe

In [ ]:
cleaned_dataframe = clean_data(df)
# Вывод результатов
display(cleaned_dataframe)

In [ ]:
# Распределение данных по спискам
# Можно использовать регулярные выражения для распарсивания строк
# Функция для распарсивания строк
def parse_strings(series):
    # Регулярное выражение для поиска контактных данных
    pattern = r'(\S+)'  # (\S+) для версии Python >= 3.7
    results = []
    for item in series:
        matches = re.findall(pattern, item)
        results.extend(matches)
    return results

# Распределение данных по спискам
df['phone'] = parse_strings(df['phone'])
df['email'] = parse_strings(df['email'])
df['website'] = parse_strings(df['website'])

print(df)

In [ ]:
# Вывод топа 10 гостиниц по количеству телефонов и отзывов
top_phones = grouped_data.sort_values(['phone', 'platform'], ascending=[False, False]).head(10)
top_reviews = grouped_data.sort_values(['review_count', 'platform'], ascending=[False, False]).head(10)

In [ ]:
# Вычисление квадрата координат размером 1 км на 1 км, где больше всего гостиниц
# Это может потребовать дополнительной логики для определения границ квадрата

In [ ]:
# Задача со звездочкой
# Здесь можно реализовать различные сценарии объединения данных, например:
# merged_data = pd.merge(df1, df2, on='uid', how='left').fillna(method='ffill')
# top_merged = merged_data.sort_values(['review_count', 'platform'], ascending=[False, False]).head(10)

In [ ]:
# Вывод результатов
display(grouped_data)
display(top_phones)
display(top_reviews)